In [7]:
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import json
import cv2 as cv
import os 
import numpy as  np 

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

# Create a torch dataset

In [6]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224, 0.225])

        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]
        img = cv.cvtColor(cv.COLOR_BGR2RGB)
        img = self.transform(img)
        kps = np.array(item['kps']).flatten()
        kps = np.astype(kps, np.float32)
        ## map the position of keypoints after resizing
        kps[::2] *= 224.0 / w
        kps[1::2] *= 224.0 / h

        return img, kps

In [ ]:
train_dataset = KeypointsDataset('data/images', "data/data_train.json")
val_dataset = KeypointsDataset('data/images', 'data/data_val.json')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create A Model

In [ ]:
model = models.resnet50(pretrained=True)

model.fc = torch.nn.Linear(model.fc.in_features, 14 * 2) # Repalces the last layer


In [ ]:
model = model.to(device)


# Train model

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs = 20

for epoch in range(epochs):
    runing_loss = 0.0
    model.train()
    for image, kps in train_loader:
        image = image.to(device)
        kps = kps.to(device)
        output = model(image)
        loss = criterion(output, kps)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runing_loss += loss.item()
    print(f'Training Loss :{runing_loss / len(train_loader) : .4f}')
    with torch.no_grad():
        model.eval()
        val_loss = 0.0
        for image, kps in val_loader : 
            image = image.to(device)
            kps = kps.to(device)
            output = model(image)
            loss_v = criterion(output, kps)
            val_loss += loss_v.item()
        print(F"val loss : {val_loss / len(val_loader) :.4f}")

            



